In [ ]:
#$ export FLASK_APP=endpoints.py
#$ flask run

In [1]:
import requests
from sql_caller import engine_connector, user_call
from sqlalchemy import text
import pandas as pd

In [55]:
res = {"admin_nick":"Pelayo","group_name":"Atleti","recv1_nick":"Arda"}

if len(res) > 2 and 'admin_nick' in res and 'group_name' in res:
    q_columns = ['group_name', 'user_id_admin', 'user_1_id', 'user_2_id', 'user_3_id']
    q_len = len(res)

    q_columns_str = ', '.join(q_columns[:q_len])

    group_name = res.pop('group_name')
    users_ids = [user_call(x) for x in res.values()]
    comillas = lambda x: "'"+str(x)+"'"
    comillado = list(map(comillas, users_ids))
    q_values_str = ', '.join(comillado)

    conn = engine_connector()

    query = f"""INSERT INTO chat_api.users_has_groups ({q_columns_str})
                    VALUES ('{group_name}', {q_values_str});
                """
    conn.execute(query)

IntegrityError: (MySQLdb._exceptions.IntegrityError) (1364, "Field 'user_2_id' doesn't have a default value")
[SQL: INSERT INTO chat_api.users_has_groups (group_name, user_id_admin, user_1_id)
                    VALUES ('Atleti', '5', '6');
                ]
(Background on this error at: http://sqlalche.me/e/13/gkpj)

In [40]:
user_call('Pelayo')

5

In [ ]:
comillas = lambda x: "'"+x+"'"
    comillado = list(map(comillas, q_result.values()))
    q_values_str = ', '.join(comillado)

In [ ]:
@app.route("/")
def hello_world():
    return {'hello':'world'}

@app.route('/name')
def te_doy_tu_nombre():
    name = request.args.get('name')
    lastname = request.args.get('lastname')
    return {"Hello": f"{name} {lastname}"}

@app.route('/mucho')
def mucho():
    args = request.args
    res = {}
    for k, v in args.items():
        res[k] = v
    return res

